## 导入库

In [1]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from torch.autograd import Variable

## 获取数据

In [2]:
trans_composes = torchvision.transforms.Compose([
    torchvision.transforms.Resize(28),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5],[0.5])
])

In [3]:
train_dataset = torchvision.datasets.MNIST("mnist_data",train=True,transform=trans_composes,download=True)

In [4]:
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)

### 查看数据

In [5]:
len(train_loader)

1875

In [6]:
img, label = next(iter(train_loader))

In [7]:
print(img.shape,label)

torch.Size([32, 1, 28, 28]) tensor([0, 0, 8, 9, 6, 9, 6, 5, 4, 6, 4, 7, 7, 1, 7, 2, 5, 2, 8, 8, 1, 0, 0, 6,
        8, 1, 6, 9, 5, 5, 6, 5])


In [8]:
image_size = (1,28,28)

In [9]:
# random noises dim
latent_dim = 100

## 生成器

In [10]:
class generator(nn.Module):
    def __init__(self):
        super(generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim,64),
            nn.ReLU(inplace=True),
            nn.Linear(64,128),
            nn.BatchNorm1d(128, 0.8),
            nn.ReLU(inplace=True),
            nn.Linear(128,256),
            nn.BatchNorm1d(256, 0.8),
            nn.ReLU(inplace=True),
            nn.Linear(256,512),
            nn.BatchNorm1d(512, 0.8),
            nn.ReLU(inplace=True),
            nn.Linear(512,1024),
            nn.BatchNorm1d(1024, 0.8),
            nn.ReLU(inplace=True),
            nn.Linear(1024,int(np.prod(image_size))),
            nn.Tanh()
        )
    def forward(self,z):
        # shape of z: [batchsize, laten_dim]
        output = self.model(z)
        image = output.reshape(output.shape[0],*image_size)
        
        return image

## 判别器

In [11]:
class discriminator(nn.Module):
    def __init__(self):
        super(discriminator,self).__init__()
        self.model = nn.Sequential(
            nn.Linear(int(np.prod(image_size)),1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024,512),
            nn.ReLU(inplace=True),
            nn.Linear(512,256),
            nn.ReLU(inplace=True),
            nn.Linear(256,128),
            nn.ReLU(inplace=True),
            nn.Linear(128,1),
            nn.Sigmoid(),
        )
    def forward(self,image):
        # shape of image: [batchsize, 1, 28, 28]
        # output is probably
        prob = self.model(image.reshape(image.shape[0],-1))
        return prob

### 使用gpu/cpu

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [13]:
Tensor = torch.cuda.FloatTensor if device else torch.FloatTensor
Tensor

torch.cuda.FloatTensor

## 构建超参数

In [14]:
# Initialize generator
G = generator()
g_optimizer = torch.optim.Adam(G.parameters(),lr=0.0002,betas=(0.5,0.999))

In [15]:
# Initialize discriminator
D = discriminator()
d_optimizer = torch.optim.Adam(D.parameters(),lr=0.0002,betas=(0.5,0.999))

In [16]:
# Initialize args
# def init_weights(m):
#         if type(m) == nn.Linear:
#             nn.init.xavier_uniform_(m.weight)
# G.apply(init_weights)
# D.apply(init_weights)

In [17]:
# loss function
loss_fn = nn.BCELoss()

In [18]:
# use device
G.to(device)
D.to(device)
loss_fn.to(device)

BCELoss()

In [19]:
num_epoch = 20
batch_size = 32

## 训练

In [24]:
# epoch numbers
for epoch in range(num_epoch):
    total_g_loss = 0.0
    total_d_loss = 0.0
    # only need image,no use label
    for i,(imgs,_) in enumerate(train_loader):
        # true image label:1
        valid = Variable(Tensor(imgs.size(0),1).fill_(1.0),requires_grad=False)
        # fake image label:0
        fake = Variable(Tensor(imgs.size(0),1).fill_(0.0),requires_grad=False)
        
        # real_image
        real_image = Variable(imgs.type(Tensor))
        
        # gen_image
        # train generator
        g_optimizer.zero_grad()
        # generator random noises
        z = Variable(Tensor(
            np.random.normal(0,1,(imgs.shape[0],latent_dim))
        ))
        # generator fake image
        gen_image = G(z)
        # generaor loss:make discriminator error
        g_loss = loss_fn(D(gen_image),valid)
        # gradient descent,update args
        g_loss.backward()
        g_optimizer.step()
        
        # train discriminator
        d_optimizer.zero_grad()
        # real image loss
        real_loss = loss_fn(D(real_image),valid)
        # fake image loss
        fake_loss = loss_fn(D(gen_image.detach()),fake)
        # the mean of real iamge and fake image
        d_loss = real_loss+fake_loss
        # gradient descent and update args
        d_loss.backward()
        d_optimizer.step()
#         print(
#             "[D loss: %f] [G loss: %f]"
#             % (d_loss.item(), g_loss.item())
#         )
        total_d_loss += d_loss
        total_g_loss += g_loss
    print("epoch:{}, g_loss:{},d_loss:{}".format(epoch+1,total_g_loss.item(),total_d_loss.item()))

[D loss: 0.760875] [G loss: 1.625079]
[D loss: 0.776660] [G loss: 1.735653]
[D loss: 0.917835] [G loss: 1.469765]
[D loss: 1.071759] [G loss: 1.428819]
[D loss: 0.722857] [G loss: 1.517761]
[D loss: 0.734444] [G loss: 1.637611]
[D loss: 0.938878] [G loss: 1.496275]
[D loss: 0.839379] [G loss: 1.403787]
[D loss: 0.797666] [G loss: 1.388430]
[D loss: 0.857912] [G loss: 1.368140]
[D loss: 0.828004] [G loss: 1.514430]
[D loss: 0.701043] [G loss: 1.560397]
[D loss: 0.969081] [G loss: 1.599490]
[D loss: 0.741001] [G loss: 1.583696]
[D loss: 0.883016] [G loss: 1.362822]
[D loss: 1.016472] [G loss: 1.574749]
[D loss: 0.651479] [G loss: 1.417154]
[D loss: 0.750450] [G loss: 1.443109]
[D loss: 0.785593] [G loss: 1.421734]
[D loss: 0.834163] [G loss: 1.443322]
[D loss: 0.591033] [G loss: 1.560572]
[D loss: 0.692166] [G loss: 1.569144]
[D loss: 0.706793] [G loss: 1.408259]
[D loss: 1.130300] [G loss: 1.424680]
[D loss: 0.863933] [G loss: 1.526588]
[D loss: 0.759748] [G loss: 1.651278]
[D loss: 0.7

[D loss: 0.659533] [G loss: 1.440372]
[D loss: 0.846262] [G loss: 1.529022]
[D loss: 0.919827] [G loss: 1.481073]
[D loss: 0.725185] [G loss: 1.585068]
[D loss: 0.851882] [G loss: 1.619965]
[D loss: 1.032439] [G loss: 1.214063]
[D loss: 0.548891] [G loss: 1.727959]
[D loss: 0.735434] [G loss: 1.507069]
[D loss: 0.842670] [G loss: 1.529307]
[D loss: 0.799185] [G loss: 1.491189]
[D loss: 0.853735] [G loss: 1.431814]
[D loss: 0.738747] [G loss: 1.554512]
[D loss: 0.678845] [G loss: 1.563904]
[D loss: 0.799182] [G loss: 1.561727]
[D loss: 0.889197] [G loss: 1.438356]
[D loss: 0.718956] [G loss: 1.584556]
[D loss: 0.754086] [G loss: 1.399008]
[D loss: 1.022797] [G loss: 1.348977]
[D loss: 0.625731] [G loss: 1.618219]
[D loss: 0.671767] [G loss: 1.698970]
[D loss: 0.727515] [G loss: 1.527528]
[D loss: 0.792094] [G loss: 1.483293]
[D loss: 0.704076] [G loss: 1.514463]
[D loss: 0.630744] [G loss: 1.794778]
[D loss: 0.763910] [G loss: 1.575302]
[D loss: 1.034060] [G loss: 1.475943]
[D loss: 0.6

[D loss: 0.773046] [G loss: 1.530064]
[D loss: 0.834925] [G loss: 1.747313]
[D loss: 0.856621] [G loss: 1.334038]
[D loss: 0.649806] [G loss: 1.261252]
[D loss: 0.838709] [G loss: 1.543360]
[D loss: 0.691607] [G loss: 1.695953]
[D loss: 0.668939] [G loss: 1.757574]
[D loss: 0.758928] [G loss: 1.738251]
[D loss: 0.798282] [G loss: 1.299993]
[D loss: 0.787334] [G loss: 1.540055]
[D loss: 0.956524] [G loss: 1.675674]
[D loss: 1.018682] [G loss: 1.783654]
[D loss: 0.751777] [G loss: 1.513803]
[D loss: 0.906567] [G loss: 1.225568]
[D loss: 0.811762] [G loss: 1.408052]
[D loss: 0.856737] [G loss: 1.402449]
[D loss: 0.559318] [G loss: 1.647343]
[D loss: 0.881065] [G loss: 1.529432]
[D loss: 0.887951] [G loss: 1.427026]
[D loss: 0.939617] [G loss: 1.577906]
[D loss: 0.696803] [G loss: 1.559899]
[D loss: 1.256428] [G loss: 1.548713]
[D loss: 0.757646] [G loss: 1.324991]
[D loss: 0.872702] [G loss: 1.541808]
[D loss: 0.915174] [G loss: 1.496535]
[D loss: 0.927728] [G loss: 1.565568]
[D loss: 0.7

[D loss: 0.867533] [G loss: 1.384007]
[D loss: 0.697557] [G loss: 1.603045]
[D loss: 0.483140] [G loss: 1.664975]
[D loss: 1.020409] [G loss: 1.515819]
[D loss: 0.750459] [G loss: 1.718884]
[D loss: 0.762041] [G loss: 1.752352]
[D loss: 0.973368] [G loss: 1.581989]
[D loss: 0.663274] [G loss: 1.553560]
[D loss: 0.952380] [G loss: 1.479711]
[D loss: 0.983918] [G loss: 1.572048]
[D loss: 0.973941] [G loss: 1.516973]
[D loss: 0.812414] [G loss: 1.340935]
[D loss: 0.881857] [G loss: 1.450160]
[D loss: 0.997529] [G loss: 1.402948]
[D loss: 0.748333] [G loss: 1.410392]
[D loss: 0.782754] [G loss: 1.317241]
[D loss: 0.800560] [G loss: 1.481839]
[D loss: 0.661881] [G loss: 1.518357]
[D loss: 1.019906] [G loss: 1.790421]
[D loss: 0.944440] [G loss: 1.560969]
[D loss: 0.983912] [G loss: 1.566552]
[D loss: 0.980111] [G loss: 1.352165]
[D loss: 0.793889] [G loss: 1.330423]
[D loss: 0.836606] [G loss: 1.282535]
[D loss: 0.750434] [G loss: 1.450505]
[D loss: 0.624455] [G loss: 1.491000]
[D loss: 0.7

[D loss: 0.625375] [G loss: 1.295236]
[D loss: 0.988775] [G loss: 1.477642]
[D loss: 0.635123] [G loss: 1.613852]
[D loss: 0.528071] [G loss: 1.398475]
[D loss: 0.770056] [G loss: 1.605959]
[D loss: 0.813327] [G loss: 1.683865]
[D loss: 0.669225] [G loss: 1.514287]
[D loss: 1.181628] [G loss: 1.342181]
[D loss: 0.613840] [G loss: 1.485290]
[D loss: 0.822113] [G loss: 1.568959]
[D loss: 0.619930] [G loss: 1.431235]
[D loss: 0.711028] [G loss: 1.438923]
[D loss: 0.874092] [G loss: 1.851031]
[D loss: 0.710137] [G loss: 1.634499]
[D loss: 0.810618] [G loss: 1.397678]
[D loss: 0.776510] [G loss: 1.567293]
[D loss: 0.685295] [G loss: 1.665758]
[D loss: 0.633749] [G loss: 1.589665]
[D loss: 0.805794] [G loss: 1.516067]
[D loss: 0.750404] [G loss: 1.765681]
[D loss: 0.613333] [G loss: 1.540720]
[D loss: 0.783533] [G loss: 1.372576]
[D loss: 0.718674] [G loss: 1.678149]
[D loss: 0.844730] [G loss: 1.653630]
[D loss: 1.065562] [G loss: 1.619960]
[D loss: 1.309400] [G loss: 1.197565]
[D loss: 0.7

...

[D loss: 0.874036] [G loss: 1.289240]
[D loss: 0.868709] [G loss: 1.478552]
[D loss: 0.944267] [G loss: 1.495003]
[D loss: 0.871691] [G loss: 1.443710]
[D loss: 0.836039] [G loss: 1.429796]
[D loss: 0.594510] [G loss: 1.491264]
[D loss: 0.756139] [G loss: 1.468074]
[D loss: 0.619946] [G loss: 1.542650]
[D loss: 0.959591] [G loss: 1.422892]
[D loss: 0.854668] [G loss: 1.542307]
[D loss: 0.907253] [G loss: 1.262273]
[D loss: 1.088239] [G loss: 1.389142]
[D loss: 0.910586] [G loss: 1.494876]
[D loss: 0.890986] [G loss: 1.387625]
[D loss: 0.586742] [G loss: 1.598194]
[D loss: 0.863852] [G loss: 1.349269]
[D loss: 1.100667] [G loss: 1.225563]
[D loss: 0.723872] [G loss: 1.241567]
[D loss: 0.867732] [G loss: 1.457038]
[D loss: 0.663306] [G loss: 1.592426]
[D loss: 0.643006] [G loss: 1.677545]
[D loss: 0.670819] [G loss: 1.475553]
[D loss: 0.753030] [G loss: 1.498120]
[D loss: 0.780639] [G loss: 1.602193]
[D loss: 0.866002] [G loss: 1.512087]
[D loss: 0.745480] [G loss: 1.445033]
[D loss: 0.7

[D loss: 0.703009] [G loss: 1.575052]
[D loss: 0.903865] [G loss: 1.526128]
[D loss: 0.780556] [G loss: 1.375117]
[D loss: 0.706581] [G loss: 1.342430]
[D loss: 0.659126] [G loss: 1.481650]
[D loss: 0.718758] [G loss: 1.582165]
[D loss: 0.873798] [G loss: 1.764009]
[D loss: 0.900485] [G loss: 1.617661]
[D loss: 0.909588] [G loss: 1.638303]
[D loss: 1.125015] [G loss: 1.240899]
[D loss: 0.739324] [G loss: 1.323563]
[D loss: 1.009740] [G loss: 1.537728]
[D loss: 0.827790] [G loss: 1.374907]
[D loss: 1.042254] [G loss: 1.611800]
[D loss: 0.886169] [G loss: 1.229633]
[D loss: 1.035734] [G loss: 1.407950]
[D loss: 0.824241] [G loss: 1.417754]
[D loss: 0.816131] [G loss: 1.452808]
[D loss: 0.974008] [G loss: 1.586407]
[D loss: 0.871551] [G loss: 1.498817]
[D loss: 0.888447] [G loss: 1.642612]
[D loss: 0.737463] [G loss: 1.514969]
[D loss: 0.647022] [G loss: 1.319035]
[D loss: 0.873684] [G loss: 1.291119]
[D loss: 0.812407] [G loss: 1.390264]
[D loss: 0.854004] [G loss: 1.344795]
[D loss: 0.7

[D loss: 0.809815] [G loss: 1.502659]
[D loss: 0.545490] [G loss: 1.455767]
[D loss: 0.809927] [G loss: 1.487325]
[D loss: 0.771582] [G loss: 1.486719]
[D loss: 0.756904] [G loss: 1.368843]
[D loss: 0.778777] [G loss: 1.425544]
[D loss: 1.046881] [G loss: 1.500309]
[D loss: 0.797706] [G loss: 1.589436]
[D loss: 0.820969] [G loss: 1.661465]
[D loss: 1.000189] [G loss: 1.539147]
[D loss: 0.716475] [G loss: 1.543975]
[D loss: 0.726581] [G loss: 1.608324]
[D loss: 0.809897] [G loss: 1.317783]
[D loss: 0.684458] [G loss: 1.392242]
[D loss: 0.664925] [G loss: 1.748765]
[D loss: 0.844471] [G loss: 1.583441]
[D loss: 0.815143] [G loss: 1.389791]
[D loss: 0.717743] [G loss: 1.676905]
[D loss: 0.702547] [G loss: 1.515205]
[D loss: 1.016383] [G loss: 1.352847]
[D loss: 0.778806] [G loss: 1.387667]
[D loss: 0.682026] [G loss: 1.649974]
[D loss: 0.950736] [G loss: 1.584795]
[D loss: 0.917974] [G loss: 1.348497]
[D loss: 0.586456] [G loss: 1.566743]
[D loss: 0.964865] [G loss: 1.613200]
[D loss: 0.7

[D loss: 1.003102] [G loss: 1.271435]
[D loss: 0.761988] [G loss: 1.231106]
[D loss: 0.761173] [G loss: 1.240430]
[D loss: 0.795628] [G loss: 1.412004]
[D loss: 0.994762] [G loss: 1.616379]
[D loss: 1.124455] [G loss: 1.313943]
[D loss: 0.696761] [G loss: 1.452135]
[D loss: 0.614091] [G loss: 1.509021]
[D loss: 0.796980] [G loss: 1.580463]
[D loss: 0.718979] [G loss: 1.567792]
[D loss: 0.793866] [G loss: 1.487664]
[D loss: 0.947194] [G loss: 1.452679]
[D loss: 0.804459] [G loss: 1.440256]
[D loss: 0.591990] [G loss: 1.416389]
[D loss: 0.854329] [G loss: 1.485846]
[D loss: 0.815689] [G loss: 1.362370]
[D loss: 0.746857] [G loss: 1.458553]
[D loss: 0.888612] [G loss: 1.413411]
[D loss: 0.867972] [G loss: 1.815552]
[D loss: 0.984971] [G loss: 1.689798]
[D loss: 0.868367] [G loss: 1.626531]
[D loss: 0.745941] [G loss: 1.276085]
[D loss: 0.952259] [G loss: 1.387099]
[D loss: 0.806277] [G loss: 1.571183]
[D loss: 0.906574] [G loss: 1.344087]
[D loss: 0.754843] [G loss: 1.508711]
[D loss: 0.7

[D loss: 0.921828] [G loss: 1.485921]
[D loss: 0.752081] [G loss: 1.397077]
[D loss: 0.562890] [G loss: 1.483294]
[D loss: 0.841833] [G loss: 1.405160]
[D loss: 0.661876] [G loss: 1.500688]
[D loss: 0.793272] [G loss: 1.277819]
[D loss: 1.099249] [G loss: 1.405992]
[D loss: 0.821283] [G loss: 1.369401]
[D loss: 1.071697] [G loss: 1.516780]
[D loss: 1.012444] [G loss: 1.467178]
[D loss: 1.070923] [G loss: 1.574413]
[D loss: 0.733868] [G loss: 1.553588]
[D loss: 0.851477] [G loss: 1.383924]
[D loss: 0.904410] [G loss: 1.330988]
[D loss: 0.751352] [G loss: 1.534686]
[D loss: 0.817088] [G loss: 1.256067]
[D loss: 0.987490] [G loss: 1.482011]
[D loss: 0.791845] [G loss: 1.338148]
[D loss: 0.962094] [G loss: 1.495779]
[D loss: 0.781825] [G loss: 1.467132]
[D loss: 0.828866] [G loss: 1.437512]
[D loss: 0.755314] [G loss: 1.682248]
[D loss: 0.893806] [G loss: 1.270350]
[D loss: 0.743218] [G loss: 1.419915]
[D loss: 0.831322] [G loss: 1.389137]
[D loss: 1.122896] [G loss: 1.262034]
[D loss: 0.9

## 测试图像生成效果

In [28]:
z = Variable(Tensor(
    np.random.normal(0,1,(imgs.shape[0],latent_dim))
))
# generator fake image
gen_image = G(z)

In [30]:
toPIL = torchvision.transforms.ToPILImage()
pic = toPIL(gen_image[0])
pic.save('result.jpg')